In [1]:
# Essentials
import time
import copy
from collections import OrderedDict
import random
import os
from tifffile import TiffFile
from PIL import Image, ImageOps
from pathlib import Path
from tqdm.notebook import tqdm
# Sklearn
from sklearn.model_selection import train_test_split
# Data
import numpy as np
import pandas as pd
# Plot
import matplotlib.pyplot as plt
# Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, Subset
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
# Torchvision
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
# Local 
from unet import UNet
from LCD import LandCoverData
from dataset import *
from train import *

# Dataset

In [2]:
LCD = LandCoverData()

In [39]:
print(unet)

UNet(
  (encoder1): Sequential(
    (enc1conv1): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu1): ReLU(inplace=True)
    (enc1conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu2): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder2): Sequential(
    (enc2conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc2relu1): ReLU(inplace=True)
    (enc2conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

### Define custom transforms

The motivation behind redefining transforms is that we need to apply the same transform to both mask and image.

In [38]:
train_dir = 'Small_dataset/train'
test_dir = 'Small_dataset/test'

train_set, val_set = train_val_dataset(ImageSegementationDataset(train_dir), val_split=0.2)
test_set = ImageSegementationDataset(test_dir, mode='test')

print("Train set contains", len(train_set), "elements")
print("Validation set contains", len(val_set), "elements")
print("Test set contains", len(test_set), "elements")

Train set contains 16 elements
Validation set contains 4 elements
Test set contains 34 elements


In [9]:
batch_size = 16
epochs= 10
lr= 0.001

loader_train = DataLoader(train_set, batch_size=batch_size, shuffle=True)
loader_valid = DataLoader(val_set, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(test_set, batch_size=batch_size, shuffle=True)

data_sizes = {"train": len(loader_train), "valid": len(loader_valid)}
print(data_sizes)

{'train': 1, 'valid': 1}


In [6]:
# Adam optimizer
unet = UNet()
optimizer = torch.optim.Adam(unet.parameters(), lr=lr)

In [7]:
# Class_weight
weights = np.zeros((LCD.N_CLASSES,))
num_ign_classes = len(LCD.IGNORED_CLASSES_IDX)
weights[num_ign_classes:] = (1 / LCD.TRAIN_CLASS_COUNTS[2:])* LCD.TRAIN_CLASS_COUNTS[2:].sum() / (LCD.N_CLASSES-2)
weights[LCD.IGNORED_CLASSES_IDX] = 0.

class_weights = torch.FloatTensor(weights)

# Criterion
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [156]:
class DiceLoss(nn.Module):

    def __init__(self):
        super(DiceLoss, self).__init__()
        self.smooth = 1.0

    def forward(self, y_pred, y_true):
        assert y_pred.size() == y_true.size(), "Both vectors must have the same shape"
        y_pred = y_pred[:, 0].contiguous().view(-1)
        y_true = y_true[:, 0].contiguous().view(-1)
        intersection = (y_pred * y_true).sum()
        dsc = (2. * intersection + self.smooth) / (
            y_pred.sum() + y_true.sum() + self.smooth
        )
        return 1. - dsc

In [223]:
def training(model, train_loader, valid_loader, data_sizes, epochs, optimizer, scheduler, title):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Device", device)
    model.to(device)
    
    since = time.time()

    training_loss = []
    validation_loss = []
    num_workers = 1

    best_model = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    loaders = {"train": train_loader, "valid": valid_loader}
    print(data_sizes)
    step = 0

    #class_weights = class_weight().to(device)
    #criterion = nn.CrossEntropyLoss(class_weights)
    criterion = dice_coef_multilabel
    for epoch in range(1, epochs+1):
        print(f'Epoch {epoch}/{epochs}')
        print('-' * 10)

        for phase in ["train", "valid"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_iou = 0
            running_kl_div = 0

            for image, mask in tqdm(loaders[phase]):
                image = image.to(device)
                mask = mask.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase=='train'):

                    output = model(image)
                    # print(output)
                    _, preds = torch.max(output, 1)

                    #loss = criterion(output, torch.tensor(mask, dtype=torch.long, device=device).squeeze())
                    # print(preds.size())
                    # print(torch.argmax(output, axis=1).shape)
                    loss = criterion(output, mask.squeeze(), LCD.N_CLASSES)
                

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item()
                # running_corrects += torch.sum(iou_pytorch(output, mask))
                running_iou += mIOU(mask, preds)
                running_kl_div += epsilon_kl_divergence(mask.cpu(), preds.cpu())

            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss/data_sizes[phase]
            epoch_iou = running_iou/data_sizes[phase]
            epoch_kl = running_kl_div/data_sizes[phase]
            if phase == 'train':
                training_loss.append(epoch_loss)
            else:
                validation_loss.append(epoch_loss)

            print('{} Loss: {:.4f} IoU: {:.4f} KL_div: {:.4f}'.format(phase, epoch_loss, epoch_iou, epoch_kl))
            
            if phase == 'valid' and epoch_iou > best_acc:
                best_acc = epoch_iou
                best_model = copy.deepcopy(model.state_dict())

            
    # Plotting the validation loss and training loss
    print('validation loss: ' + str(validation_loss))
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # load best model weights
    model.load_state_dict(best_model)

    # plot the training and validation loss
    plt.figure()
    plt.plot(training_loss, 'b', label='Training Loss')
    plt.plot(validation_loss, 'r', label='Validation Loss')
    plt.title(title)
    plt.legend()
    plt.show() #Change title for every model

    return model

In [224]:
def train_model(data_dir, model, epochs):
    
    class_weights = torch.FloatTensor(weights)

    # Optimizing all parameters
    optimizer_ft = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    #optimizer_ft = optim.Adam(model.parameters(), lr = 1e-3)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

    # Training the model
    title = 'Variations of the training and validation loss'
    model_ft = training(model, loader_train, loader_valid, data_sizes, epochs, optimizer_ft, exp_lr_scheduler, title)

    return model_ft

In [225]:
def epsilon_kl_divergence(y_true, y_pred):
    class_distribution_true = np.apply_along_axis(np.bincount, axis=1, arr=y_true.flatten(1), minlength=LCD.N_CLASSES)
    class_distribution_pred = np.apply_along_axis(np.bincount, axis=1, arr=y_pred.flatten(1), minlength=LCD.N_CLASSES)
    # Normalize to sum to 1  
    normalized_class_distribution_true = (class_distribution_true.T/class_distribution_true.sum(1)).T
    normalized_class_distribution_pred = (class_distribution_pred.T/class_distribution_pred.sum(1)).T
    # add a small constant for smoothness around 0
    normalized_class_distribution_true += 1e-7
    normalized_class_distribution_pred += 1e-7

    score = np.mean(np.sum(normalized_class_distribution_true * np.log(normalized_class_distribution_true / normalized_class_distribution_pred), 1))
    try:
        assert np.isfinite(score)
    except AssertionError as e:
        raise ValueError('score is NaN or infinite') from e
    return score

In [244]:
def dice_coef(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = (y_pred_f * y_true_f).sum()
    smooth = 0.0001
    return (2. * intersection + smooth) / (y_true_f.sum() + y_pred_f.sum() + smooth)

def change_shape(y_true, y_pred, numLabels):  

    encoded_target = y_pred.data.clone().zero_()
    encoded_target[...] = 0
    print(torch.tensor(y_true.unsqueeze(1), dtype=torch.int64))
    encoded_target.scatter_(1, torch.tensor(y_true.unsqueeze(1), dtype=torch.int64), 1.)
    encoded_target = Variable(encoded_target)

    return encoded_target

def dice_coef_multilabel(y_pred, y_true, numLabels):
    dice=0
    y_true = change_shape(y_true, y_pred, numLabels)
    # print(y_true.size())
    # print(y_pred.size())
    for index in range(numLabels):
        dice += dice_coef(y_true[:,index,:,:], y_pred[:,index,:,:])
    return dice/numLabels # taking average

In [245]:
data, mask = next(iter(loader_train))
random_mask = 2*torch.rand(size=(16,10,256,256))-1
random_mask = torch.sigmoid(random_mask)

In [247]:
rain_model('dataset/train', unet, epochs=10)

# Pre-trained Model

In [19]:
import segmentation_models_pytorch as smp

unet_pre_trained = smp.Unet(encoder_name='resnet18',in_channels=4, classes=10, activation='softmax')

In [20]:
train_model('dataset/train', unet_pre_trained, epochs=10)

Device cpu
{'train': 1, 'valid': 1}
Epoch 1/10
----------


torch.Size([12, 256, 256])
tensor([[[4, 3, 3,  ..., 4, 4, 4],
         [4, 8, 8,  ..., 9, 9, 4],
         [4, 8, 8,  ..., 8, 4, 4],
         ...,
         [4, 8, 1,  ..., 8, 1, 1],
         [4, 3, 4,  ..., 8, 3, 3],
         [6, 8, 8,  ..., 4, 8, 3]],

        [[4, 4, 1,  ..., 4, 4, 4],
         [4, 8, 7,  ..., 4, 3, 4],
         [4, 8, 3,  ..., 7, 7, 4],
         ...,
         [4, 3, 8,  ..., 8, 1, 1],
         [4, 4, 4,  ..., 4, 4, 1],
         [6, 8, 7,  ..., 7, 8, 8]],

        [[4, 3, 4,  ..., 3, 8, 4],
         [4, 7, 8,  ..., 3, 3, 4],
         [4, 8, 4,  ..., 3, 8, 1],
         ...,
         [3, 8, 8,  ..., 8, 8, 3],
         [4, 4, 4,  ..., 1, 8, 3],
         [6, 8, 6,  ..., 8, 8, 8]],

        ...,

        [[4, 4, 1,  ..., 3, 3, 4],
         [4, 8, 8,  ..., 3, 4, 4],
         [4, 8, 8,  ..., 3, 4, 4],
         ...,
         [4, 8, 8,  ..., 0, 8, 9],
         [4, 4, 4,  ..., 7, 4, 8],
         [6, 8, 8,  ..., 3, 8, 6]],

        [[0, 4, 4,  ..., 4, 4, 4],
         [4, 8, 8,  

IndexError: too many indices for tensor of dimension 3

In [113]:
torch.save(unet_pre_trained.state_dict(), "unet-resnet18.pt")

In [ ]:
trained_unet = train_model('dataset/train', unet_pre_trained, epochs=10)

In [ ]:
unet_pre_trained_bis = smp.Unet(encoder_name='vgg11',in_channels=4, classes=10, activation='softmax')

In [ ]:
trained_unet_bis = train_model('dataset/train', unet_pre_trained_bis, epochs=10)

# Saving the model

In [ ]:
torch.save(model_ft.state_dict(),"unet.pt")

# Loading the model

In [ ]:
def loading_saved_model(model_name):
    """Loads the saved model"""
    model = unet
    model.load_state_dict(torch.load(model_name, map_location = device))
    model.eval()
    return model

model_loaded = loading_saved_model("unet.pt")

# Prediction of the model

In [ ]:
column_names = ['no_data', 'clouds', 'artificial', 'cultivated', 'broadleaf', 'coniferous', 'herbaceous', 'natural', 'snow', 'water']

ids_test = np.arange(10)

def compute_class_counts(masks, n_classes):

    dist = np.zeros((masks.size(0), n_classes))

    for i, mask in enumerate(masks):
        arr = mask.numpy()
        count = np.bincount(arr.ravel(), minlength=n_classes)
        count[0], count[1] = 0, 0
        dist[i] = count/np.sum(count)
    return dist

def getting_pixel_distribution(model, test_loader, test_size):

    indx_test = np.arange(10087,10087+test_size, dtype=np.int32).reshape(-1,1)
    all_dist = np.zeros((test_size, LCD.N_CLASSES))

    for counter, images in enumerate(tqdm(test_loader)):
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        distributions = compute_class_counts(preds, LCD.N_CLASSES)
        all_dist[counter*images.size(0):(counter+1)*images.size(0)] = distributions

    df = pd.DataFrame(all_dist, columns=column_names)
    df.insert(0, 'Sample_id', range(10087, 10087 + len(df)))

    return df

In [ ]:
test_dir = 'dataset/test'
test_set = ImageSegementationDataset(test_dir, mode='test')
test_loader = DataLoader(test_set, batch_size=batch_size)
test_size = len(test_set)

df = getting_pixel_distribution(model_loaded, test_loader, test_size)

In [ ]:
def create_from_pandas_csv(df):
    df.to_csv(r'results.csv', index = False)

create_from_pandas_csv(df)